In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation


#1.1 Create the Dataset and Split it into Training and Validation Sets
BATCH_SIZE = 32
IMG_SIZE = (150, 150)

train_dataset = image_dataset_from_directory("dataset/training", 
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42
                                            )

test_data = image_dataset_from_directory("dataset/test",
                                        shuffle=True,
                                        batch_size=BATCH_SIZE,
                                        image_size=IMG_SIZE,
                                        validation_split=0.2,
                                        subset='validation',
                                        seed=42
                                        )

layer = keras.layers.Dense(3)
layer.build((None, 4))  # Create the weights

print("weights:", len(layer.weights))
print("trainable_weights:", len(layer.trainable_weights))
print("non_trainable_weights:", len(layer.non_trainable_weights))

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 400 files belonging to 2 classes.
Using 80 files for validation.
weights: 2
trainable_weights: 2
non_trainable_weights: 0


2022-05-31 11:45:45.323049: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.



In [6]:
base_model.trainable = False

In [7]:
inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. 
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [8]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.fit(train_dataset, epochs=10, validation_data=test_data)

Epoch 1/10
50/50 [==============================] - 41s 773ms/step - loss: 0.5895 - binary_accuracy: 0.8306 - val_loss: 0.1308 - val_binary_accuracy: 0.9625
Epoch 2/10
50/50 [==============================] - 39s 788ms/step - loss: 0.2320 - binary_accuracy: 0.9219 - val_loss: 0.0913 - val_binary_accuracy: 0.9750
Epoch 3/10
50/50 [==============================] - 40s 793ms/step - loss: 0.1579 - binary_accuracy: 0.9381 - val_loss: 0.0776 - val_binary_accuracy: 0.9625
Epoch 4/10
50/50 [==============================] - 39s 776ms/step - loss: 0.1373 - binary_accuracy: 0.9525 - val_loss: 0.0841 - val_binary_accuracy: 0.9625
Epoch 5/10
50/50 [==============================] - 39s 784ms/step - loss: 0.1018 - binary_accuracy: 0.9606 - val_loss: 0.1140 - val_binary_accuracy: 0.9500
Epoch 6/10
50/50 [==============================] - 40s 794ms/step - loss: 0.0939 - binary_accuracy: 0.9669 - val_loss: 0.0651 - val_binary_accuracy: 0.9625
Epoch 7/10
50/50 [==============================] - 39s 77